# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'study16' : "5TtKaKCouyJp7Hhtu4YlYm",
    'study17' : "5qX1Tq3IQ74iSgrlMfRhty",
    'study18' : "1bwXMYoRgEFWebWZ9ZPBqs",
    'study19' : "3ackEOD2vox5Oc1vjeALKJ", 
    'study20' : "3Cmpe4nGzOQDefowW49pBS",
    'PianoJz' : "2IqcxvGty13ewL583z9Kdn",
    'cringe1' : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study17'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100



HTTP Error for PUT to https://api.spotify.com/v1/playlists/5qX1Tq3IQ74iSgrlMfRhty/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 1, 100-200 --> 298:
	{'snapshot_id': 'AAADBCx7AdzcCxOdyjJA7ucRB6q/5Nqx'}
ERROR, 400-406 --> 185:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/5qX1Tq3IQ74iSgrlMfRhty/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 3, 100-200 --> 285:
	{'snapshot_id': 'AAADBeU7HuTleTC1zNLBlBTUZE9gPb9j'}
Iteration 4, 200-300 --> 14:
	{'snapshot_id': 'AAADBlRlq/G+UlWcbGMSyweJsqDlvRDu'}

Phase 2: Chunks of 10

Iteration 1, 330-340 --> 358:
	{'snapshot_id': 'AAADB8sqGSAPrY3TXa921yWvW4gZojsm'}
Iteration 2, 50-60 --> 141:
	{'snapshot_id': 'AAADCJy6pv1h/DYBcN8GLlDcAeS4jJFl'}
Iteration 3, 0-10 --> 294:
	{'snapshot_id': 'AAADCffq0QjALCIo1WU1X6jgbKAhFeAK'}
Iteration 4, 50-60 --> 200:
	{'snapshot_id': 'AAADCv4FYVe1JwvyDcQgPhq8VJGnTRCI'}
Iteration 5, 40-50 --> 52:
	{'snapshot_id': 'AAADC17DwWarphh0JvZcYCIR/21l4QTt'}
Iteration 6, 100-110 --> 224:
	{'snapshot_id': 'AAADDIvp4d4E70KUW2ZDw6R6MvJ+17yQ'}
Iteration 7, 290-300 --> 152:
	{'snapshot_id': 'AAADDU

HTTP Error for PUT to https://api.spotify.com/v1/playlists/5qX1Tq3IQ74iSgrlMfRhty/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 29, 70-80 --> 332:
	{'snapshot_id': 'AAADI6TdemyapHqnVZ/sWjRyAeCkGXvS'}
ERROR, 400-406 --> 22:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/5qX1Tq3IQ74iSgrlMfRhty/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 31, 340-350 --> 162:
	{'snapshot_id': 'AAADJKm3NXwcDSeu6IuhU2PvCT1rwMET'}
Iteration 32, 160-170 --> 45:
	{'snapshot_id': 'AAADJf2AgxVLDZ5kbIRG8VEibKMZii/g'}
Iteration 33, 220-230 --> 291:
	{'snapshot_id': 'AAADJiIWBifSwDO8WYJOYNiLddSEqObI'}
Iteration 34, 380-390 --> 114:
	{'snapshot_id': 'AAADJzTe+Sq2B+qLfQFM7vHRq3NNHGbg'}
Iteration 35, 290-300 --> 43:
	{'snapshot_id': 'AAADKB/SnK3CE62eDjnLWosEHyLFf6FY'}
Iteration 36, 90-100 --> 301:
	{'snapshot_id': 'AAADKTHs0whisxIYVjSYBW2xch4HwIwq'}
Iteration 37, 370-380 --> 221:
	{'snapshot_id': 'AAADKs0kvoNtCvpwh1YtiCMRHLA4koT/'}
Iteration 38, 340-350 --> 55:
	{'snapshot_id': 'AAADK6bU/UAEFqlESpeevlAGJ2FWMoYf'}
Iteration 39, 210-220 --> 283:
	{'snapshot_id': 'AAADLEBVsNuWCQ0m

HTTP Error for PUT to https://api.spotify.com/v1/playlists/5qX1Tq3IQ74iSgrlMfRhty/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 32, 50-55 --> 175:
	{'snapshot_id': 'AAADTcfozsVUxnJyzIjb+Y3lBwLmm8Uo'}
ERROR, 405-406 --> 340:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/5qX1Tq3IQ74iSgrlMfRhty/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 34, 320-325 --> 377:
	{'snapshot_id': 'AAADTtYhs09ZSgR1ouAdBoSc0C1iGiS0'}
Iteration 35, 380-385 --> 32:
	{'snapshot_id': 'AAADT/pY6uIZClScqsj/fvk+BMufL30m'}
Iteration 36, 120-125 --> 319:
	{'snapshot_id': 'AAADUOy7yq9uqBd52Gl6tizawYO7682a'}
Iteration 37, 365-370 --> 247:
	{'snapshot_id': 'AAADUWmqYKH6CQsfZDdzziZMjh4KyWG4'}
Iteration 38, 40-45 --> 275:
	{'snapshot_id': 'AAADUgQNeXSam+a7WPzFsGy82R4YjI/0'}
Iteration 39, 55-60 --> 197:
	{'snapshot_id': 'AAADUyV+YcTHTLxZTQP8xHkeT13t8d8d'}
Iteration 40, 325-330 --> 141:
	{'snapshot_id': 'AAADVMJ/wo+bA2qFt1NwiXmjGyrF0aB2'}
Iteration 41, 300-305 --> 331:
	{'snapshot_id': 'AAADVTDIysKhos3VudW6VYiVlmHOo5Xe'}

Phase 3: 1 at a Time, 1/3

Iteration 1, 184-185 --> 245:
	{'snap